# Autenticidad, 
# los Mensajes Secretos,
# y Seguridad



<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/master/04_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparar el Espacio (Prepare the Space)

In [9]:
! pip install eciespy eth_keys gspread oauth2client
! git clone https://github.com/ProfDoeg/Colegio_Invisible.git
! pip install --upgrade gspread

Cloning into 'Colegio_Invisible'...
remote: Enumerating objects: 610, done.
remote: Counting objects: 100% (424/424), done.
remote: Compressing objects: 100% (399/399), done.
remote: Total 610 (delta 293), reused 23 (delta 23), pack-reused 186
Receiving objects: 100% (610/610), 35.03 MiB | 16.43 MiB/s, done.
Resolving deltas: 100% (405/405), done.


# Historias

<img src="https://github.com/ProfDoeg/Colegio_Invisible/raw/main/img/barb_2.jpeg" width="250"/>


*¡Verdezuela, Verdezuela,\
Suéltame tu cabellera!*\
-La Bruja

*Encryption is a powerful defensive\
weapon for free people.*\
-Esther Dyson




# Aplicar Criptografía Para Crear un Ensamblaje de Mensajería Seguro 
# (Apply Cryptography to Create a Secure Messaging Assembly)

## Esquema (Outline) 

Aplicar criptografía para crear un tablero de mensajes securos.
***
Apply cryptography to build a secure messaging board.

## Metas (Goals)

- crear y guardar claves\
(create and save keys)
- descargar/cargar claves\
(download/upload keys)
- descifrar el token de acceso a la hoja de cálculo\
(decrypt spreadsheet access token)
- entender `gspread`\
(understand `gspread`)
- publicar: nombre, clave pública, mensaje público y firma\
(post: name, public key, public message, and signature)
- firmar mensaje\
(sign message)
- verificar mensaje\
(verify message)
- escribir mensaje a otros participantes\
(write message to other participants)
- escribir un mensaje encriptado a otro participante\
(write encrypted message to other participant)
- escribir mensaje firmado y encriptado\
(write signed and encrypted message)


# Gestionar Archivos de Claves (Manage Key Files)

## Obtener Claves (Get Keys)

3 Opciones (3 Options):
- hacer nuevas claves\
(make new keys)
- cargar claves generadas previamente\
(load previously generated keys)
  - desde la disco local\
  (from local drive)
  - desde Google Drive\
  (from Google Drive)


### Hacer Nuevas Claves (Make New Keys)

In [3]:
! mkdir llaves

In [4]:
! python Colegio_Invisible/scripts/ecc_generate.py llaves/prvkey1.eck

Input password for encrypting keyfile:
Ingresar password para cifrar el archivo de la clave:
Repetir (Repeat): 

Passwords match...

Passwords son iguales...
Generación de la clave privada está terminada (Private key generation complete)
La clave privada está cifrada y escrita en un archivo binario:
Private key encrypted and written to binary file: llaves/prvkey1.eck
cb9b7dc1a693039e6390a4e4f2d3f6d84ecf218cdc45c20aafcf4e4ea0a3a7fcbc21a34f0883dc3f050f71f14cef629c0cd85eaa3dc7f6d43ff711dc8c201d60


In [5]:
! python Colegio_Invisible/scripts/ecc_pubkey_extract.py llaves/prvkey1.eck llaves/pubkey1.eck 

Input password for private key file:
Ingresar password para acceder al archivo del la clave privada:
Input password for public key file:
Ingresar password para acceder al archivo del la clave pública: 
Repetir (Repeat): 

Passwords match...

Passwords son iguales...
Extracción de la clave pública terminada desde
(Completed public key extraction from): llaves/prvkey1.eck
La clave pública está cifrada y escrita en archivo binario
(Public key encrypted and written to binary file): llaves/pubkey1.eck
62cab2c31c9a13f8e912ba14e1706e0f0b15de2b1ea0d6026e65611d8e651a20d9b31241dcac3362b4e5f0d47c594f9980fc45f312f11e749b26c15756f762a967f8f152c5680482517d92df458c976ee175977f023294308e7738591b675f10


### Cargar Claves (Load Keys)

**EJECUTAR SOLO SI HAS GUARDADO PREVIAMENTE LAS CLAVES**
***
**ONLY RUN IF YOU HAVE PREVIOUSLY SAVED KEYS**

#### De Disco Local (From Local Disk)

In [ ]:
from google.colab import files
fu=files.upload();

In [ ]:
! unzip llaves.zip

#### De Google Drive (From Google Drive)

1. Montar Google Drive\
(Mount Google Drive)
2. Copiar el archivo de clave desde la ubicación al directorio actual\
(Copy key file from location to present directory)
3. Unzip archivo de clave\
(Unzip key file)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp drive/MyDrive/cinv/llaves.zip llaves.zip 
! unzip llaves.zip

### Guardar Llaves (Save Keys) 

#### A Disco Local (To Local Disk)

In [ ]:
#note
! zip -r llaves.zip llaves

In [ ]:
#note
from google.colab import files
f=files.download('llaves.zip');

#### A Google Drive (To Google Drive)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp llaves.zip drive/MyDrive/cinv/llaves.zip 

# Usar Guion Para Acceder Clave Pública (Use Script to Access Public Key)

In [10]:
ret=!{ 'python Colegio_Invisible/scripts/ecc_keydump.py llaves/pubkey1.eck'}
pubkey1=ret[-1]
pubkey1

'7da1b363b5942d7aeeeedb092520701b44f12e39d6c2972edd11821ad1db4f8c137f604db9b099fc37f41d8033d66f16b93b234760c1ba6bf05850d6b2917965'

# Gestionar el Acceso a Google Sheets (Manage Access to Google Sheets)


## Descifrar Credenciales de API (Decrypt API Credentials)

$\color{white}{\text{c0leg101nv1s1ble}}$

Usar la contraseña de arriba para descifrar `client_secret.aes`
***
Use the above password to decrypt `client_secret.aes`





In [11]:
! python Colegio_Invisible/scripts/aes_decrypt.py Colegio_Invisible/secrets/client_secret.aes Colegio_Invisible/secrets/client_secret.json 

Input password for decryption:
Ingresar password para decifrar: 
Éxito (Success): Colegio_Invisible/secrets/client_secret.aes decifrado con éxito (decryption complete)
Escrito a (Written to): Colegio_Invisible/secrets/client_secret.json


In [12]:
! ls Colegio_Invisible/secrets

client_secret.aes  client_secret.json


## `gspread` 



`gspread` es un módulo de Python para la interacción programática con Google Sheets.
***
`gspread` is a python module for programmatic interaction with Google Sheets. 
***
***
Todos los participantes tendrán acceso a la misma hoja de cálculo en la nube.
***
All the participants will have access to the same spreadsheet in the cloud.

https://docs.gspread.org/en/latest/user-guide.html

Usar las credenciales descifradas para autenticar y obtener acceso API a la hoja de cálculo
***
Use the decrypted credentials to authenticate and gain API access to the spreadsheet

In [13]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('Colegio_Invisible/secrets/client_secret.json', scope)
client = gspread.authorize(creds)
sheets = client.open("signatures")

# Sheets

Aquí podemos ver la estructura de la hoja de cálculo y todas las capas o `Worksheet`
***
Here we can see the structure of the spreadsheet and all of the layers or `Worksheet`

In [14]:
sheets.worksheets()

[<Worksheet 'Multi' id:0>,
 <Worksheet 'Magic' id:2092809244>,
 <Worksheet 'ChorusSignature' id:549527758>,
 <Worksheet 'StealProposal' id:858983463>,
 <Worksheet 'StealSignature' id:475665302>,
 <Worksheet 'test' id:2104752728>]

## Sheet 0 : Roster

Aislar la primera hoja donde consultamos la primera fila
***
Isolate the first sheet where we query the first row

In [15]:
sheet=sheets.worksheets()[0]
sheet.row_values(1)

['name', 'pubkey', 'statement', 'signature']

**PELIGRO: NO EJECUTAR**
***
**DANGER: DO NOT RUN**

In [ ]:
#sheet.clear()
#sheet.update_cell(1, "name")
#sheet.append_row(["name", "pubkey","statement","signature"])
#sheet.__dir__()

### Publicar en la Lista (Publish in the Roster)

Publicar (Publish):

- `name` of participant\
(nombre)
- `pubkey` public key of participant in hex\
(clave público de participante en hexadecimal)
- `statement` is the declaration signed by participant\
(la declaración firmada por el participante)
- `signature` is the ecc signature of the statement\
(es la firma ecc de la declaración)

#### Entrada Datos (Input Data)

In [18]:
my_name=input('input name')
my_statement=input('input statement')
open('llaves/statement1.txt','wb').write(my_statement.encode())

input nameSeco
input statementI am Seco


9

In [19]:
! cat llaves/statement1.txt

I am Seco

Usar script para firmar la declaración
***
Use script to sign statement

In [22]:
ret = !{ 'python Colegio_Invisible/scripts/ecc_sign.py llaves/prvkey1.eck llaves/statement1.txt llaves/sig1.sig'}
sig1=ret[-1]
sig1

'66689eb5107196fac83544bf25627018707ad1badddfdf3cf9b9e57c584ce8974ddd2f6ccad1a2e309fefc3004a1fea3c39b7fa77dffdcaf983efcce7b1ee40d00'

#### Publicar Datos (Publish Data)


In [20]:
sheet.append_row([my_name,pubkey1,my_statement,sig1])

{'spreadsheetId': '1z_iLwzlqa_JX7AdRR3OsBeFEgvS0jVYh1oo-X23bMOY',
 'tableRange': 'Multi!A1:D6',
 'updates': {'spreadsheetId': '1z_iLwzlqa_JX7AdRR3OsBeFEgvS0jVYh1oo-X23bMOY',
  'updatedCells': 4,
  'updatedColumns': 4,
  'updatedRange': 'Multi!A7:D7',
  'updatedRows': 1}}

## Leer Sheet 0 (Read Sheet 0)

In [23]:
import pandas as pd

dataframe = pd.DataFrame(sheet.get_all_records())
dataframe

,name,pubkey,statement,signature
0,DrDoeg,aa403c5cc3bd3f8970f4b69a19b19f758a01b8b7df51dc...,I am DrDoeg,2671da66309b34b396f42983350e213205f0510124058d...
1,lil late,d4870b35f981df8a87545c0d4e3879c52417e3c914cbf4...,das aright,2c10c648ea437d08e33f0a28a83c5ca56986d23b3e046f...
2,Paulina,f7ee21788227541ac7d9d684fed40f1c89772ebba67027...,I am Paulina R,346d7e2fad7856a8b86ef1435fb328284ef123668c39a2...
3,Lau,cbf0666e1f02013145d325d813aaaa5f0ffd49361136cf...,Soy Lau,7eb257548a943314a2818a1a9f649c6cfc1d9c475e598b...
4,Pamela,74bb3d46084d27eb4341b5e3531b16e8911bb6fb765aa9...,Soy Pamela,f68597821580b15584f42574b812d3d3b4b043ffb70dbe...
5,Seco,7da1b363b5942d7aeeeedb092520701b44f12e39d6c297...,I am Seco,Fallada (Fail) [Errno 2] No such file or direc...


We can verify statements. Here we use the `ecies` cryptography library directly without using the external scripts. 

In [ ]:
import eth_keys
import ecies

eth_keys.datatypes.Signature(bytes.fromhex(sig1)).verify_msg(my_statement.encode(),eth_keys.keys.PublicKey(bytes.fromhex(pubkey1)))

In [ ]:
def verify(sig,statement,pubkey):
  return eth_keys.datatypes.Signature(bytes.fromhex(sig)).verify_msg(statement.encode() if type(statement)!=bytes else statement,eth_keys.keys.PublicKey(bytes.fromhex(pubkey)))

In [ ]:
verify(sig1,my_statement,pubkey1)

Here we verify all rows of the roster

In [ ]:
dataframe.set_index('name').apply(lambda row: verify(row.signature,row.statement,row.pubkey),axis=1)

Collecting the names of all the participants

In [ ]:
participants=dataframe.name.tolist()
participants

Doing variable assignment for needed Worksheets

In [ ]:
sheet1=sheets.worksheets()[1]
sheet2=sheets.worksheets()[2]
sheet3=sheets.worksheets()[3]
sheet1,sheet2,sheet3

In [ ]:
#ONLY FOR DDOG MC
#df=pd.DataFrame('dada',columns=participants,index=participants)
#sheet1.clear()
#sheet1.update([df.columns.values.tolist()] + df.values.tolist())
#sheet2.clear()
#sheet2.update([df.columns.values.tolist()] + df.values.tolist())
#sheet3.clear()
#sheet3.update([df.columns.values.tolist()] + df.values.tolist())
#df

## Worksheet 1

This is a place for casual public chats
- row is sender
- column is recipient

In [ ]:
dataframe1 = pd.DataFrame(sheet1.get_all_records())
dataframe1.index=dataframe1.columns
dataframe1

Make a little recipient widget interface

In [ ]:
import ipywidgets as widgets
w=widgets.Dropdown(
    options=participants,
    value='DrDoeg',
    description='Recipient:',
    disabled=False,
)

In [ ]:
#have a look
w

In [ ]:
w.value

In [ ]:
my_name

Creating messaging

In [ ]:
p_dict=dict([ (e,i) for i,e in enumerate(participants)])
p_dict

Select your message recipient using the widget.

In [ ]:
print(f'You are {my_name}')
print('Select a message recipient')
w

Write message 

Post to spreadsheet

refresh dataframe from worksheet

In [ ]:
message=input('send a message')
write_cell=(p_dict[my_name]+2,p_dict[w.value]+1)
sheet1.update_cell(*write_cell, message)
dataframe1 = pd.DataFrame(sheet1.get_all_records())
dataframe1.index=dataframe1.columns
dataframe1

In [ ]:
dataframe1 = pd.DataFrame(sheet1.get_all_records())
dataframe1.index=dataframe1.columns
dataframe1

In [ ]:
dataframe1[my_name]

# Encrypted Messaging

In [ ]:
dataframe2 = pd.DataFrame(sheet2.get_all_records())
dataframe2.index=dataframe2.columns
dataframe2

In [ ]:
def encrypt_message(hexPubKey,message):
    if type(message)!=bytes:
        message=message.encode()
    return ecies.encrypt(hexPubKey,message).hex()

In [ ]:
w

In [ ]:
encrypt_message(dataframe.set_index('name').pubkey[w.value],'hey guys hey hey')

In [ ]:
message=input('send an encrypted message')
write_cell=(p_dict[my_name]+2,p_dict[w.value]+1)
sheet2.update_cell(*write_cell, encrypt_message(dataframe.set_index('name').pubkey[w.value],message))
dataframe2 = pd.DataFrame(sheet2.get_all_records())
dataframe2.index=dataframe2.columns
dataframe2

In [ ]:
dataframe2 = pd.DataFrame(sheet2.get_all_records())
dataframe2.index=dataframe2.columns
dataframe2

In [ ]:
def import_privKey(path,password):
  import hashlib
  if type(password)!=bytes:
    password=password.encode()
  f=open(path,'rb')
  decrypted_bytes=ecies.aes_decrypt(key=hashlib.sha256(password).digest(),cipher_text=f.read())
  privKey=eth_keys.keys.PrivateKey(decrypted_bytes)
  f.close()
  return privKey

def decrypt_message(privKey,cipherhex):
    ciphertext=bytes.fromhex(cipherhex)
    try:
      return ecies.decrypt(privKey.to_hex(),ciphertext)
    except:
      return None

In [ ]:
decrypt_message(import_privKey('llaves/prvkey1.eck',''),dataframe2[my_name][my_name])


In [ ]:
dataframe2 = pd.DataFrame(sheet2.get_all_records())
dataframe2.index=dataframe2.columns
dataframe2

k=import_privKey('llaves/prvkey1.eck','')
tr=dataframe2[my_name].apply(lambda x: decrypt_message(k,x))
tr

In [ ]:
def sign_message(privKey,message):
    if type(message)!=bytes:
        message=message.encode()
    signature= privKey.sign_msg(message)
    return signature.to_bytes().hex()

In [ ]:
mess=input('message')
mess_sig=sign_message(import_privKey('llaves/prvkey1.eck',''),mess)
mess_sig

In [ ]:
emess=encrypt_message(pubkey1,mess)
emess

In [ ]:
emess_sig=sign_message(k,bytes.fromhex(emess))
emess_sig,len(emess_sig)

In [ ]:
emess_sig,emess

In [ ]:
def sign_encrypt_message(privKey,hexpubKey,message):
    if type(message)!=bytes:
        message=message.encode()
    byte_enc=bytes.fromhex(encrypt_message(hexpubKey,message))
    signature= privKey.sign_msg(byte_enc)
    return signature.to_bytes().hex()+byte_enc.hex()

In [ ]:
sgen=sign_encrypt_message(k,pubkey1,mess)
sgen

In [ ]:
def decrypt_verify(pubkey,privkey,block):
  try:
    ver=verify(block[:130],bytes.fromhex(block[130:]),pubkey)
    return decrypt_message(privkey,block[130:]),ver
  except:
    return None

In [ ]:
decrypt_verify(pubkey1,k,sgen)

In [ ]:
message=input('send a signed and encrypted message')
write_cell=(p_dict[my_name]+2,p_dict[w.value]+1)
signed_enc_message=sign_encrypt_message(k,pubkey1,message)
sheet3.update_cell(*write_cell, encrypt_message(dataframe.set_index('name').pubkey[w.value],signed_enc_message))
dataframe3 = pd.DataFrame(sheet3.get_all_records())
dataframe3.index=dataframe3.columns
dataframe3

In [ ]:
dataframe3 = pd.DataFrame(sheet3.get_all_records())
dataframe3.index=dataframe3.columns
dataframe3

In [ ]:
pubkey_name=dataframe.set_index('name')['pubkey']
dataframe3[[my_name]].reset_index().apply(lambda x: decrypt_verify(pubkey_name[x['index']],k,x[my_name])  ,axis=1)